In [1]:
import os, time
import requests
import random
import pickle
import spacy
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from pprint import pprint

import dash
import dash_table as dt
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import plotly.express as px

from PIL import Image
import glob

apikey = os.getenv('GUARDIAN_APIKEY')

## Scraping the inputs

Re-purpose the code from dataset creation to scrape just 5 new articles, this time from any section of the Guardian.

In [2]:
# Helper functions
def get_results(base_url, params):
    
    results = []
    try:
        r = requests.get(base_url, params)
        r.raise_for_status()
    except requests.exceptions.RequestException as err:
        raise SystemExit(err)
    
    data = r.json()
    results.extend(data['response']['results'])
    return results


def results_to_html(results):
    
    # grab urls, write to file
    urls = [result['webUrl'] for result in results]

    # retrieve HTML from urls
    html_files = {}
    while len(html_files) < len(urls): 
        
        for i, url in enumerate(urls):
            if i not in html_files:
                try:
                    file = requests.get(url, timeout=0.5)
                    file.raise_for_status()
                    html_files[i] = file
                except requests.exceptions.RequestException as err:
                    # print(err)
                    time.sleep(2)
    
    return html_files
        
        
def html_to_text(html_files):
    
    all_texts = []
    
    for file_id, file in html_files.items():
        soup = BeautifulSoup(file.content, 'html.parser')
        body = soup.find_all('div', class_='article-body-commercial-selector')
        if len(body) == 1:
            ps = body[0].find_all('p')
        par_list = [p.text for p in ps]
        text = " ".join(par_list)
        text = text.replace('\xa0', ' ')
        if not (text == ''):
            all_texts.append(text)
        
    return all_texts

# Main function to scrape and collect news articles for inference
def get_text():
    
    API_ENDPOINT = "http://content.guardianapis.com/search"
    # pick a random page, to display different results
    page_number = random.randint(0,15)
    my_params = {
        'api-key': apikey,
        'order-by': 'relevance', 
        'from-date': "2020-1-1",
        'page-size': 10,
        'page': page_number
    }
    
    results = get_results(API_ENDPOINT, my_params)
    html_files = results_to_html(results)
    texts = html_to_text(html_files)
    df = pd.DataFrame({'Content': texts})
    
    return df

In [3]:
df = get_text()
df

,Content
0,All adults in the UK should have been offered ...
1,Emerging from Holy Family hospital in New Delh...
2,A consortium that includes high street giant N...
3,Labour has urged ministers to make May’s elect...
4,There was a sense of excitement at the Garden ...
5,Three in four Australians agree that Scott Mor...
6,The public will not see Donald Trump’s White H...
7,The Department of Home Affairs mistakenly sent...
8,Vogue will publish a limited print edition of ...
9,Companies that make dangerous building materia...


## From inputs to category prediction

Load trained classifier and TF-IDF vectoriser.

In [4]:
with open("models/best_svm.pickle", 'rb') as model:
    svc = pickle.load(model)
    
with open("processed/tfidf_vectoriser.pickle", 'rb') as f:
    vectoriser = pickle.load(f)
    
nlp = spacy.load("en_core_web_sm")

Create dictionary to convert predictions.

In [5]:
# create dictionary
sections = ['environment', 'business', 'film', 'culture', 'education']
d = {i: section for i, section in enumerate(sections)}

# add the "other" category when model is unsure
d.update({5: "not sure"})
print(d)

{0: 'environment', 1: 'business', 2: 'film', 3: 'culture', 4: 'education', 5: 'not sure'}


In [6]:
# Helper functions
def keep_token(t):
    """Decide whether to keep a token"""
    return (t.is_alpha and not (t.is_space or t.is_punct or t.is_stop))


def lemmatised_string(doc):
    """Lemmatise remaining tokens"""
    return " ".join(t.lemma_ for t in doc if keep_token(t))


def predict_category(features):
    initial_preds = svc.predict(features)
    probs = svc.predict_proba(features).max(axis=1)
    preds = []
    
    for prob, in_pred in zip(probs, initial_preds):
        if prob > 0.5:
            preds.append(in_pred)
        else:
            preds.append(5)
    return [d[pred] for pred in preds], probs

# Main function 
def clean_text_and_predict(df_0):
    
    df = df_0.copy()
    df['Content_parsed'] = df['Content'].str.lower()
    df['Content_parsed'] = df['Content_parsed'].str.strip()
    
    # parse and clean articles
    docs = list(nlp.pipe(df['Content_parsed'], disable=['tok2vec','ner','tagger','parser']))
    df['Content_parsed'] = [lemmatised_string(doc) for doc in docs]
    
    # numericalise with learnt transformer
    features = vectoriser.transform(df['Content_parsed']).toarray()
    
    preds, probs = predict_category(features)
    df['Prediction'] = preds
    df['Confidence'] = probs
    df['Confidence'] = (df['Confidence']*100).map('{:,.0f}%'.format)
    
    return df

In [7]:
clean_text_and_predict(df)

,Content,Content_parsed,Prediction,Confidence
0,All adults in the UK should have been offered ...,adult uk offer covid vaccination september dom...,business,74%
1,Emerging from Holy Family hospital in New Delh...,emerge holy family hospital new delhi ram verm...,not sure,50%
2,A consortium that includes high street giant N...,consortium include high street giant think pol...,business,99%
3,Labour has urged ministers to make May’s elect...,labour urge minister election england covid se...,environment,64%
4,There was a sense of excitement at the Garden ...,sense excitement garden house devon weekend ga...,culture,68%
5,Three in four Australians agree that Scott Mor...,australians agree scott morrison publicly rebu...,culture,76%
6,The public will not see Donald Trump’s White H...,public donald trump white house record year gr...,culture,84%
7,The Department of Home Affairs mistakenly sent...,department home affair mistakenly send outdate...,business,71%
8,Vogue will publish a limited print edition of ...,vogue publish limit print edition february iss...,culture,54%
9,Companies that make dangerous building materia...,company dangerous build material grenfell towe...,business,80%


## Create app in Dash

In [8]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

colors = {
    'background': '#FFFFFF',#'#212F3D',
    'text': '#000000' #'#FBFCFC'
}

#current_dir = os.getcwd() + "/images/"

markdown = """
This app predicts the topic of different bodies of text, either input by the user or news articles scraped from the Guardian.
"""

app.layout = html.Div(style={'backgroundColor': colors['background']}, children=[   

    # title
    html.H1(
        children='Wanna see if I can guess...',
        style={
            'textAlign': 'center',
            'color': colors['text']
        }
    ),
    
    # space
    html.Br(),
    
    # main section
    html.Div([
        
        # LEFT PANEL (prediction on user input)
        html.Div([
            html.H3("...what you are writing about?"),
            
            dcc.Textarea(id='input-state',
                         placeholder='Write here',
                         style={'width': 500, 'height': 120}),
            
            html.Div([
                html.Button(id='submit-button-1', n_clicks=0, children='Let me guess',
                        style={'marginBottom':50})
            ]),
                       
            html.H5(id='output-1'),
            
            html.Br(),
            
            html.Div(id='image-1', style={'marginTop': 100})
        ],
        style={'width': '45%', 'display': 'inline-block', 'textAlign': 'center', 'verticalAlign': 'top'}),
        
        
        # RIGHT PANEL (prediction on scraped news)
        html.Div([
            html.H3("...what the Guardian is writing about?"),
            
            html.Button(id='submit-button-2', n_clicks=0, children='Fetch some news'),
            
            dcc.Graph(id='graph-2'),
            
            dt.DataTable(id='table',
                         columns=[{'name':'Content', 'id':'Content'},
                                  {'name':'Prediction', 'id':'Prediction'},
                                  {'name':'Confidence', 'id':'Confidence'}],
                         page_size=5,
                         style_data={'whiteSpace':'normal',
                                     'height':'auto'},
                         style_cell={'textAlign':'left'}
            )],
            style={'width': '45%', 'display': 'inline-block', 'textAlign': 'center', 'verticalAlign': 'top'}),

    ]),
    
])

# callbacks

@app.callback(
    Output('output-1', 'children'),
    Output('image-1', 'children'),
    Input('submit-button-1', 'n_clicks'),
    State('input-state', 'value')
)
def predict_from_input(n_clicks, input_value):
    
    if n_clicks == 0:
        return f"Try me! I'll surprise ya.", ''
    else:
        df = pd.DataFrame({'Content': input_value}, index=[0])
        df = clean_text_and_predict(df)
        pred = df.loc[0, 'Prediction']
        fname = glob.glob(f'/Users/michele/github/Guardian_news_classification/assets/{pred}.*')[0].split('/')[-1]
        src = app.get_asset_url(f"{fname}")
        img = html.Img(src=src, style={'width':'60%', 'height':'60%'})
        if pred == 'not sure':
            return "I'm not sure this time...", img
        else:
            return f"Are you perhaps talking about {pred}?", img

#
@app.callback(
   Output('table', 'data'),
   Output('graph-2', 'figure'),
   Input('submit-button-2', 'n_clicks')
)
def scrape_and_predict(n_clicks):
    
    if n_clicks == 0:
        return [{}], px.pie(values=[1], names=[''], hole=.3)
    
    # get dataframe
    df = get_text()
    df = clean_text_and_predict(df)

    # update table
    df['Content'] = df['Content'].str.slice(stop=150)
    df['Content'] = df['Content'].astype(str) + '...'
    records = df[['Content', 'Prediction', 'Confidence']].to_dict('records')
    
    # update pie chart
    counts = df['Prediction'].value_counts()
    names, values = list(zip(*[(name, count) for name, count in counts.items()]))
    fig = px.pie(df, values=values, names=names, hole=.3)
    
    return records, fig

In [ ]:
app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
